In [ ]:
# contour between Anterior_RVIP_snap and Inferior_RVIP_snap

import cv2
### Caluclate angle between Anterior and Inferior Septum 
slc = 0 
point_ARVIP   = np.array([Anterior_RVIP[slc][0],       Anterior_RVIP[slc][1]])  # Can think of as Point A                                                                                         # Static (reference) point
point_IRVIP   = np.array([Inferior_RVIP[slc][0],       Inferior_RVIP[slc][1]])   # Can think of this as Point B                                                                                        # Static (reference) point
point_origin  = np.array([Endocardial_Centers[slc][0], Endocardial_Centers[slc][1]])                                                                                         # Origin point
vector_ARVIP  = point_ARVIP - point_origin                                                                                 # Static (reference) vector
vector_IRVIP  = point_IRVIP - point_origin 
radians       = np.arctan2(np.cross(vector_ARVIP, vector_IRVIP), np.dot(vector_ARVIP, vector_IRVIP))
Septal_Angle  = np.degrees(radians)
Mid_Septum_Angle = Septal_Angle / 2 # This is always computed as half thats why we dont need to the mid-septum cause it is always half the distance between the two there is no anaotomical reaons
Remainder_Angles = (360 - Septal_Angle) / 4



contour = Epicardium_Contours[0]
# Indices of the points ARVIP and IRVIP in the contour array
ARVIP = point_ARVIP  # Index of ARVIP in the contour array
IRVIP = point_IRVIP  # Index of IRVIP in the contour array



# Initialize the mask array
mask = np.zeros(Endocardium_Contours[0].shape, dtype=bool)
mask[ARVIP[0]:IRVIP[0],IRVIP[1]:ARVIP[1]] = True


masked_contour = mask*Endocardium_Contours[0]
plt.imshow(mask*Endocardium_Contours[0])

contour_coordinates = np.argwhere(masked_contour == 1)

################################ Version 1 using distance bewteen intermediate points
# Initialize arc length
arc_length = 0.0

# Iterate over pairs of consecutive points
for i in range(len(contour_coordinates) - 1):
    # Calculate the distance between consecutive points using the Euclidean norm
    segment_length = np.linalg.norm(contour_coordinates[i + 1] - contour_coordinates[i])
    
    # Add the segment length to the arc length
    arc_length += segment_length

print('{:.2f} pixels'.format(arc_length))

################################ Version 2 using average radius
# Calculate radii (magnitudes of vectors)
radius_ARVIP = np.linalg.norm(vector_ARVIP)
radius_IRVIP = np.linalg.norm(vector_IRVIP)

average_radius = (radius_ARVIP + radius_IRVIP) / 2.0
arc_length2 = average_radius * radians

print('{:.2f} pixels'.format(arc_length2))

##### Another way could be to minimize the angle between the 2 rather than the distance 

In [ ]:
def myocarial_contour_extraction(mask):
    import numpy as np
    import skimage
    rows = mask.shape[0] 
    cols = mask.shape[1] 
    slcs = mask.shape[2] 
    Background_Mask = np.zeros([rows, cols, slcs])
    Myocardial_Mask = np.zeros([rows, cols, slcs])
    Blood_Pool_Mask = np.zeros([rows, cols, slcs])
    Endocardial_Centers  = []
    Epicardium_Contours  = []
    Endocardium_Contours = []
    for slc in range(mask.shape[2]):
        Epicardium_Contours.append([])
        Endocardium_Contours.append([])
        Endocardial_Centers.append([])
        Background_temp = np .zeros([rows, cols])
        Myocardium_temp = np .zeros([rows, cols])
        Blood_Pool_temp = np .zeros([rows, cols])
        Mask_Label = skimage.measure.label(abs(mask[:, :, slc] - 1), 
                                           background = None, 
                                           return_num = False, 
                                           connectivity = 1)
         
        Background_temp[Mask_Label == 1] = 1
        Myocardium_temp[Mask_Label == 0] = 1
        Blood_Pool_temp[Mask_Label == 2] = 1
        
        Background_Mask[:, :, slc] = Background_temp
        Myocardial_Mask[:, :, slc] = Myocardium_temp
        Blood_Pool_Mask[:, :, slc] = Blood_Pool_temp
        
        [x_coordinates_endo,  y_coordinates_endo] = np.where(Blood_Pool_temp == 1)
        mean_y_coordinates_endo  = np.int64(np.round(np.mean(y_coordinates_endo)))
        mean_x_coordinates_endo  = np.int64(np.round(np.mean(x_coordinates_endo)))
        Endocardial_Centers[slc] = [mean_x_coordinates_endo, mean_y_coordinates_endo]
        
        epicardium_contour  = comprehensive_outter_contour(Myocardial_Mask[:, :, slc])
        endocardium_contour = comprehensive_inner_contour(Myocardial_Mask[:, :, slc], 
                                                          np.int64(Endocardial_Centers[slc][0]),
                                                          np.int64(Endocardial_Centers[slc][1]))
        Epicardium_Contours[slc]  = epicardium_contour
        Endocardium_Contours[slc] = endocardium_contour
    return [Epicardium_Contours, Endocardium_Contours, Endocardial_Centers]